In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import scipy as sp
import scanpy as sc
import pandas as pd
from tqdm import tqdm
import anndata as ad

In [3]:
df = pd.read_csv('../../data/GSE138669/SraRunTable.txt', index_col=0)

In [4]:
df.head()

,Assay Type,AssemblyName,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Center Name,chemistry,CONDITION,...,LibrarySource,Organism,Platform,ReleaseDate,create_date,version,Sample Name,source_name,SRA Study,Tissue
Run,,,,,,,,,,,,,,,,,,,,,
SRR10254548,RNA-Seq,GCF_000001405.29,98,20294765414,PRJNA576702,SAMN13002514,8453401374,GEO,10X Genomics 3 Prime V1 chemistry,CONTROL,...,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2021-05-20T00:00:00Z,2019-10-09T18:19:00Z,1,GSM4115868,"dorsal\, mid-forearm skin biopsy",SRP224991,"skin,skin"
SRR10254549,RNA-Seq,GCF_000001405.29,98,26778655722,PRJNA576702,SAMN13002513,10356538606,GEO,10X Genomics 3 Prime V1 chemistry,SSC,...,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2021-05-20T00:00:00Z,2019-10-09T18:18:00Z,1,GSM4115869,"dorsal\, mid-forearm skin biopsy",SRP224991,"skin,skin"
SRR10254550,RNA-Seq,GCF_000001405.29,98,15095378550,PRJNA576702,SAMN13002535,5144005679,GEO,10X Genomics 3 Prime V1 chemistry,CONTROL,...,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2021-05-20T00:00:00Z,2019-10-09T17:50:00Z,1,GSM4115870,"dorsal\, mid-forearm skin biopsy",SRP224991,"skin,skin"
SRR10254551,RNA-Seq,GCF_000001405.29,98,12389174896,PRJNA576702,SAMN13002534,4355503768,GEO,10X Genomics 3 Prime V1 chemistry,SSC,...,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2021-05-20T00:00:00Z,2019-10-09T18:36:00Z,1,GSM4115871,"dorsal\, mid-forearm skin biopsy",SRP224991,"skin,skin"
SRR10254552,RNA-Seq,GCF_000001405.29,98,28362510260,PRJNA576702,SAMN13002533,10214057715,GEO,10X Genomics 3 Prime V1 chemistry,CONTROL,...,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2021-05-20T00:00:00Z,2019-10-09T19:22:00Z,1,GSM4115872,"dorsal\, mid-forearm skin biopsy",SRP224991,"skin,skin"


In [3]:
# read counts
directory = '../../data/GSE138669'
adata_list = list()

for root, dirs, files in os.walk(directory):
    for filename in tqdm(files):

        if 'h5' in filename:
            adata = sc.read_10x_h5(f'../../data/GSE138669/{filename}')
            adata.var_names_make_unique()
            sc.pp.filter_cells(adata, min_genes=200)
            
            sample_name = filename.split("_")[0]
            sample_info = df[df['Sample Name'] == sample_name]
        
            adata.obs['sample'] = sample_name
            adata.obs['condition'] = sample_info['CONDITION'].values.tolist()[0]
        
            adata_list.append(adata)

100%|██████████| 24/24 [00:19<00:00,  1.23it/s]


In [4]:
adata = ad.concat(adata_list, label="batch", index_unique="_")

In [ ]:
adata

In [5]:
adata.write_h5ad('../../results/nc_2021_Tabib/scrna.h5ad')

In [6]:
adata.obs

,n_genes,sample,condition,batch
AAACATACAGAACA-1_0,1211,GSM4115868,CONTROL,0
AAACATTGTCTCAT-1_0,351,GSM4115868,CONTROL,0
AAACCGTGGGTACT-1_0,380,GSM4115868,CONTROL,0
AAACCGTGTGCTAG-1_0,960,GSM4115868,CONTROL,0
AAACGCACCCTGTC-1_0,1399,GSM4115868,CONTROL,0
...,...,...,...,...
TTTGTCAAGTGACTCT-1_21,532,GSM4115889,SSC,21
TTTGTCACAAGCCTAT-1_21,2711,GSM4115889,SSC,21
TTTGTCACACTCAGGC-1_21,2950,GSM4115889,SSC,21
TTTGTCAGTATGCTTG-1_21,1852,GSM4115889,SSC,21
